# G-Score Strategy - Investments Committee Quant Sector


In [1]:
from polygon import RESTClient
from collections import defaultdict
import csv
import statistics
import datetime
from dateutil.relativedelta import relativedelta
import ipynb.fs.defs.sec as sec
from tqdm import tqdm
import json

#not easily able to access shared files so resorting to link download instead
#from google.colab import drive
#drive.mount("/content/drive", force_remount=True)

client = RESTClient("uwQtl3txGt5BLbecq7ZbIu0ZbuitCGjc")

In [2]:
def get_g_scores(query_date: str, financials: dict):
  tickers = {}
  with open('s&p500.csv', mode ='r') as file:
    tickers = {k:v for (k,v) in csv.reader(file)} # convert csv to dict
    
  ticker_data = defaultdict(lambda: {})
  industry_data = defaultdict(lambda: defaultdict(lambda: []*13))

  try:
      client.get_aggs(ticker="AAPL", multiplier=1, timespan='day', from_=query_date, to=query_date)[0].close
  except:
    next_day = datetime.datetime.strptime(query_date, '%Y-%m-%d')
    next_day += relativedelta(days=+1)
    return get_g_scores(next_day.strftime("%Y-%m-%d"), financials)

  for t in tqdm(financials.keys()):
    try:
      earnings_per_share = sec.get_earnings_per_share(financials[t], query_date)
      shares_outstanding = sec.get_shares_outstanding(financials[t], query_date)
      debt = sec.get_total_debt(financials[t], query_date)
      cash = sec.get_cash(financials[t], query_date)
      bvps = sec.get_book_value_per_share(financials[t], query_date)
      net_income = sec.get_net_income(financials[t], query_date)
      total_assets = sec.get_total_assets(financials[t], query_date)
      ebitda = sec.get_ebitda(financials[t], query_date)
      operating_cash_flow = sec.get_operating_cash_flow(financials[t], query_date)
    
      ticker_data[t]["Price"] = client.get_aggs(ticker=t, multiplier=1, timespan='day', from_=query_date, to=query_date)[0].close
      ticker_data[t]["Volume"] = client.get_aggs(ticker=t, multiplier=1, timespan='day', from_=query_date, to=query_date)[0].volume
      ticker_data[t]["P/E"] = ticker_data[t]["Price"] / earnings_per_share
      ticker_data[t]["EV"] = shares_outstanding * ticker_data[t]["Price"]+ debt - cash
      ticker_data[t]["EBITDA"] = ebitda
      ticker_data[t]["EV/EBITDA"] = ticker_data[t]["EV"] / ticker_data[t]["EBITDA"]
      ticker_data[t]["P/B"] = ticker_data[t]["Price"] / bvps
      ticker_data[t]["ROA"] = net_income / total_assets
      ticker_data[t]["CFROA"] = operating_cash_flow / total_assets 
      ticker_data[t]["MACD"] = client.get_macd(ticker=t, timestamp = query_date).values[0].value
      ticker_data[t]["RSI"] = client.get_rsi(ticker=t, timestamp = query_date).values[0].value
      for k, v in ticker_data[t].items(): # add values to industry-specific data
        industry_data[tickers[t]][k].append(v)
    except:
      ticker_data.pop(t, None)

  print(query_date + ": " + str(len(ticker_data.keys())))

  for ind, values in industry_data.items():
    for category, data in values.items():
      industry_data[ind][category] = statistics.median(data)
  
  weightings = {"P/E": .15, "EV/EBITDA": .1, "P/B": .1, "ROA": .175, "CFROA": .175, "MACD": .15, "RSI": .15}
  comparisonTypes = {"P/E": '<', "EV/EBITDA": '<', "P/B": '<', "ROA": '>', "CFROA": '>', "MACD": '>', "RSI": '<'}
  # Dictates how ticker_val and industry_val are compared: ticker_val[P/E] < or > industry_val[P/E]

  def compare(t, category): 
    (ticker_val, industry_val) = (ticker_data[t][category], industry_data[tickers[t]][category])
    return ticker_val > industry_val if comparisonTypes[category] == '>' else ticker_val < industry_val

  g_scores = {t : sum(weight for (category, weight) in weightings.items() if compare(t, category)) 
              for t in ticker_data.keys()}
  
  with open("/Users/kevin/Documents/GitHub/g-score/output/g_scores_" + query_date + ".json", "w") as outfile:
    json.dump(g_scores, outfile)

  return g_scores

In [3]:
def backtest(g_scores_dict, starting_date):
  percent_change = 0
  curr_holdings = {}
  final_date = ''
  for curr_date in g_scores_dict.keys():
    curr_change = 0
    if datetime.datetime.strptime(curr_date, '%Y-%m-%d') != starting_date:
      for s in curr_holdings.keys():
        curr_price = client.get_aggs(ticker=s, multiplier=1, timespan='day', from_=curr_date, to=curr_date)[0].close
        curr_change += (curr_price - curr_holdings[s]) / curr_holdings[s]
      curr_change /= len(curr_holdings.keys())
    percent_change += curr_change
    curr_holdings.clear()

    num_stocks = 0
    prev_score = 0
    for stock in reversed(sorted(g_scores_dict[curr_date].items(), key=lambda x:x[1])):
      if stock[1] < prev_score and num_stocks >= 5:
        break
      curr_holdings[stock[0]] = client.get_aggs(ticker=stock[0], multiplier=1, timespan='day', from_=curr_date, to=curr_date)[0].close
      prev_score = stock[1]
      num_stocks += 1

    print("Portfolio Update: " + curr_date)
    for s in curr_holdings.keys():
      print(s + ": " + str(curr_holdings[s]))
    final_date = curr_date
  
  print("Overall P/L: " + str(percent_change * 100))
  spy_start = client.get_aggs(ticker="SPY", multiplier=1, timespan='day', from_=starting_date.strftime('%Y-%m-%d'), to=starting_date.strftime('%Y-%m-%d'))[0].close
  spy_end = client.get_aggs(ticker="SPY", multiplier=1, timespan='day', from_=final_date, to=final_date)[0].close
  print("SPY P/L: " + str((spy_end - spy_start) / spy_start * 100))

The following code runs the backtest for the last 5 years

In [4]:
tickers = {}
with open('s&p500.csv', mode ='r') as file:
    tickers = {k:v for (k,v) in csv.reader(file)} # convert csv to dict
ticker_financials = {}
for t in tickers.keys():
    try:
        ticker_financials[t] = sec.get_financials(t)
    except:
        print("Could not generate financials for " + t)

Could not generate financials for BRK.B
Could not generate financials for BF.B
Could not generate financials for FRC
Could not generate financials for SBNY
Could not generate financials for VNO


In [19]:
# create historical data
query_date = datetime.datetime.strptime("2013-03-01", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2018-02-28", '%Y-%m-%d')
while query_date <= end_date:
    get_g_scores(query_date.strftime("%Y-%m-%d"), ticker_financials)
    query_date += relativedelta(months=+1)

100%|██████████| 498/498 [02:09<00:00,  3.85it/s]


2013-03-01: 297


100%|██████████| 498/498 [01:40<00:00,  4.94it/s]


2013-04-01: 301


100%|██████████| 498/498 [01:43<00:00,  4.80it/s]


...

2017-12-01: 349


100%|██████████| 498/498 [01:48<00:00,  4.60it/s]


2018-01-02: 352


100%|██████████| 498/498 [02:00<00:00,  4.12it/s]

2018-02-01: 350


In [5]:
start_date = datetime.datetime.strptime("2013-03-01", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2023-03-01", '%Y-%m-%d')

g_scores_dict = {}
while start_date <= end_date:
    try:
        with open('/Users/kevin/Documents/GitHub/g-score/output/g_scores_' + start_date.strftime("%Y-%m-%d") + '.json') as json_file:
            data = json.load(json_file)
            g_scores_dict[start_date.strftime("%Y-%m-%d")] = data
        start_date += relativedelta(days=+1)
    except:
        start_date += relativedelta(days=+1)
print(g_scores_dict)

{'2013-03-01': {'MMM': 0.6, 'AOS': 0.7, 'ABT': 0.5, 'ACN': 0.15, 'ADM': 0.55, 'ADBE': 0.15, 'AFL': 0.85, 'A': 0.85, 'APD': 0.42500000000000004, 'ALK': 0.85, 'ALB': 0.25, 'ARE': 0.7000000000000001, 'LNT': 0.85, 'MO': 0.42500000000000004, 'AMZN': 0.5249999999999999, 'AEE': 0.5, 'AEP': 0.32499999999999996, 'AIG': 0.55, 'AMT': 0, 'AWK': 0.475, 'ABC': 0.32499999999999996, 'AME': 0.65, 'AMGN': 0.15, 'APH': 0.7, 'ADI': 0.7999999999999999, 'ANSS': 0.32499999999999996, 'AON': 0.5, 'AMAT': 0.32499999999999996, 'AJG': 0.32499999999999996, 'T': 0.575, 'AZO': 0.6, 'AVY': 0.42500000000000004, 'BAC': 0.35, 'BAX': 0.425, 'BDX': 0.75, 'BBY': 0.5, 'BIIB': 0.5, 'BLK': 0.3, 'BK': 0.5, 'BA': 0.15, 'BWA': 0.35, 'BXP': 0.85, 'BSX': 0.5, 'BMY': 0.5, 'BR': 0.42500000000000004, 'BRO': 0.7000000000000001, 'CHRW': 0.6499999999999999, 'CPB': 0.42500000000000004, 'CAH': 0.25, 'KMX': 0.15, 'CCL': 0.85, 'CE': 0.9, 'CNC': 0.35, 'CNP': 0.275, 'CF': 0.85, 'CRL': 0.75, 'SCHW': 0.1, 'CVX': 0.85, 'CB': 0.6, 'CHD': 0.1, 'CT

In [6]:
backtest(g_scores_dict, datetime.datetime.strptime("2013-03-01", '%Y-%m-%d'))

Portfolio Update: 2013-03-01
UDR: 23.97
TEL: 40.23
SWK: 77.59
CE: 46.21
PM: 91.44
Portfolio Update: 2013-04-01
WST: 31.665
TEL: 41.6
SWK: 79.83
PEP: 79.1
MKC: 36.355
KMB: 97.93
CRL: 43.48
Portfolio Update: 2013-05-01
SJM: 102.11
ODFL: 24.7
ALK: 29.865
WEC: 44.34
PFE: 28.9
MRK: 45.69
BDX: 93.58
Portfolio Update: 2013-06-03
TDY: 77.91
TEL: 44.99
SWK: 79.35
IFF: 80.29
GPC: 77.5
Portfolio Update: 2013-07-01
UNP: 77.71
ITW: 69.55
MKC: 35.27
MMC: 40.43
WDC: 63.06
WST: 35.755
WMT: 74.59
UDR: 25.57
MOS: 53.36
SJM: 104.08
TEL: 46.41
SWK: 78.35
RJF: 28.9467
PFE: 27.78
NKE: 31.165
NEE: 20.0575
NDAQ: 11.0367
MAA: 67.37
MRO: 34.83
LOW: 41.06
INTC: 23.886
HES: 66.71
BEN: 46.1867
CVX: 119.08
CF: 34.056
CCL: 34.65
APH: 19.625
LNT: 24.83
ALK: 26.425
A: 31.1803
AFL: 28.835
AOS: 18.685
Portfolio Update: 2013-08-01
CVX: 126.44
CF: 39.196
PEP: 84.2
GPC: 83.97
ED: 60.33
BDX: 103.11
Portfolio Update: 2013-09-03
WST: 37.305
UDR: 22.24
NEE: 19.9425
NDAQ: 9.9267
ITW: 71.49
CCL: 35.98
APH: 18.8375
LNT: 24.395
AF